In [ ]:
!pip install sqlalchemy
!pip install psycopg2

In [36]:
import pandas as pd

# Load the dataset
file_path = 'mock survey data 3.1.xlsx'
data = pd.read_excel(file_path, sheet_name='data')

# Handle Missing Values
data['City of Residence'] = data['City of Residence'].fillna('Unknown')
data['Purpose of Visit'] = data['Purpose of Visit'].fillna('Unknown')
data['MainHotel'] = data['MainHotel'].fillna('Unknown')

# Normalize Travel Companion Data
travel_companion_cols = [col for col in data.columns if 'Travel companion' in col]
data['TravelCompanions'] = data[travel_companion_cols].idxmax(axis=1).str.replace('Travel companion - ', '')
data.drop(columns=travel_companion_cols, inplace=True)
# print(data.isnull().sum())
data

,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,tottran,totbiz,totedu,totmedi,tototh,totshopping_rep,tot.exp,c4a_1,MainHotel,TravelCompanions
0,1.0,2018,1,India,Delhi,Leisure,Holiday/ Rest & Relax,660.046512,2.0,NaN,...,29.35,0.0,0.0,0.0,234.38,101.70,1480.90,1,15.0,Spouse
1,2.0,2018,1,India,Delhi,Leisure,Visiting friends/ relatives (who are not inter...,433.217949,2.0,NaN,...,14.09,0.0,0.0,0.0,40.00,26.70,99.10,0,Unknown,Alone
2,3.0,2018,1,United Arab Emirates,Dubai,Leisure,General business purpose,335.125000,2.0,NaN,...,242.30,0.0,0.0,0.0,0.00,4074.75,5286.27,0,Unknown,Alone
3,4.0,2018,1,India,Others,Leisure,Holiday/ Rest & Relax,490.977273,2.0,NaN,...,125.78,0.0,0.0,0.0,146.24,276.70,674.50,0,Unknown,Spouse
4,5.0,2018,1,Canada,Toronto,Business + Accompanying Pax,Visiting an international student (relative or...,379.800000,2.0,NaN,...,275.45,0.0,0.0,0.0,0.00,2070.75,4182.55,1,30.0,Alone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22969,7789.0,2018,12,Australia,Brisbane,Leisure,Holiday/ Rest & Relax,310.354839,2.0,NaN,...,260.30,0.0,0.0,0.0,225.00,59.10,1399.88,1,41.0,Spouse
22970,7790.0,2018,12,Hong Kong,Hong Kong,Leisure,Holiday/ Rest & Relax,109.393939,2.0,NaN,...,19.31,0.0,0.0,0.0,200.00,20.33,548.53,1,275.0,Friends
22971,7791.0,2018,12,Hong Kong,Hong Kong,Leisure,Holiday/ Rest & Relax,326.906250,2.0,NaN,...,20.68,0.0,0.0,0.0,130.01,7.83,454.31,1,703.0,Spouse
22972,7792.0,2018,12,Hong Kong,Hong Kong,Business + Accompanying Pax,Gather information/ facts on the education ser...,820.615385,2.0,NaN,...,86.88,0.0,0.0,0.0,0.00,7.83,297.42,0,Unknown,Alone


In [ ]:
# Preview the first few rows of both sheets to understand their contents
data_preview = excel_data.parse('data').head()
dictionary_preview = excel_data.parse('dictionary').head()

data_preview, dictionary_preview

In [ ]:
# Check for missing values and basic statistics in the data sheet to identify potential issues
data_sheet = excel_data.parse('data')
missing_values = data_sheet.isnull().sum()
data_stats = data_sheet.describe(include='all')

missing_values

In [ ]:
# Check for missing values and basic statistics in the data sheet to identify potential issues
dictionary_sheet = excel_data.parse('dictionary')
missing_values = dictionary_sheet.isnull().sum()
dictionary_stats = dictionary_sheet.describe(include='all')

missing_values

In [35]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection details
DB_HOST = "localhost:5433"
DB_NAME = "stb_db"
DB_USER = "postgres"
DB_PASS = "admin"

# Create connection engine to PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")

# Load data from Excel file
file_path = "mock survey data 3.1.xlsx"
data = pd.read_excel(file_path, sheet_name="data")
dictionary = pd.read_excel(file_path, sheet_name="dictionary")

# Normalize Travel Companion Data
travel_companion_cols = [col for col in data.columns if 'Travel companion' in col]
data['TravelCompanions'] = data[travel_companion_cols].idxmax(axis=1).str.replace('Travel companion - ', '')
data.drop(columns=travel_companion_cols, inplace=True)

# Step 1: Prepare `Visitor` table
visitor_df = data[["case", "Country of Residence", "City of Residence", "Purpose of Visit", "TravelCompanions", "Length of Stay"]].rename(
    columns={
        "case": "VisitorID",
        "Country of Residence": "Country",
        "City of Residence": "City",
        "Purpose of Visit": "Purpose",
        "Length of Stay": "LengthOfStay"
    }
).drop_duplicates()

# Step 2: Prepare `Hotel` table
# Step 2.1: Extract and Prepare the Mapping from the Dictionary
hotel_name_mapping = dictionary[dictionary['variable'] == 'MainHotel'].set_index('value')['label'].to_dict()

# Step 2.2: Extract Unique Hotel Codes from the Data
unique_hotels = data["MainHotel"].dropna().unique()

# Step 2.3: Create the Hotel Table with Mapped Names
hotel_df = pd.DataFrame({
    "HotelID": unique_hotels,
    "Name": [hotel_name_mapping.get(code, "Unknown") for code in unique_hotels],  # Map codes to names
    # "GroupName": ["Village Hotel Group"] * len(unique_hotels)  # Placeholder for group
})

# Display the resulting Hotel Table
# print(hotel_df)

# # Step 3: Prepare `Stay` table
stay_df = data[["case", "MainHotel", "year", "month", "Travel Type", "shopping_fash",
                "shopping_jewllery", "shopping_watches", "shopping_wellness", "shopping_food",
                "shopping_gifts", "shopping_consumertech", "shopping_antiques", "totfnb",
                "totacc", "tottran", "totbiz", "totedu", "totmedi", "totshopping_rep", "tot.exp"]].rename(
    columns={
        "StayID": range(1, len(data) + 1),
        "case": "VisitorID",
        "MainHotel": "HotelID",
        "year": "Year",
        "month": "Month",
        "Travel Type": "TravelType",
        "shopping_fash": "ShoppingFashionAccessories",
        "shopping_jewllery": "ShoppingJewellery",
        "shopping_watches": "ShoppingWatches",
        "shopping_wellness": "ShoppingWellness",
        "shopping_food": "ShoppingFood",
        "shopping_gifts": "ShoppingGifts",
        "shopping_consumertech": "ShoppingForTech",
        "shopping_antiques": "ShoppingAntiques",
        "totfnb": "TotalFnBSpend",
        "totacc": "TotalAccommodationSpend",
        "tottran": "TotalTransportSpend",
        "totbiz": "TotalBusinessSpend",
        "totedu": "TotalEducationSpend",
        "totmedi": "TotalMedicalSpend",
        "totshopping_rep": "TotalShoppingSpend",
        "tot.exp": "TotalExpenditure",
    }
)

# # Step 4: Prepare `GroupRevenue` table
group_revenue_data = {
    "GroupID": [98],
    "GroupName": ["Town Hotel Group"],
    "Month": [11],  # Assuming Nov as the merger month
    "Year": [2018],  # Example year
    "TotalRevenue": [500000]  # Placeholder value
}
group_revenue_df = pd.DataFrame(group_revenue_data)

# # Step 5: Prepare `Merger` table
merger_data = {
    "MergerID": [1],
    "OldHotelID": [1],  # Assuming Village Hotel Albert Court as an example
    "NewGroupID": [98],  # GroupID for Town Hotel Group
    "EffectiveDate": ["2018-11-01"]
}
merger_df = pd.DataFrame(merger_data)

# # # Step 6: Prepare `Dictionary` table
dictionary_df = dictionary.rename(columns={
    "variable": "Variable",
    "value": "Code",
    "label": "Description"
})

# # Step 7: Load data into PostgreSQL
try:
    visitor_df.to_sql("Visitor", engine, if_exists="replace", index=False)
    print("Loaded `Visitor` table.")

    hotel_df.to_sql("Hotel", engine, if_exists="replace", index=False)
    print("Loaded `Hotel` table.")

    stay_df.to_sql("Stay", engine, if_exists="replace", index=False)
    print("Loaded `Stay` table.")

    group_revenue_df.to_sql("GroupRevenue", engine, if_exists="replace", index=False)
    print("Loaded `GroupRevenue` table.")

    merger_df.to_sql("Merger", engine, if_exists="replace", index=False)
    print("Loaded `Merger` table.")

    dictionary_df.to_sql("Dictionary", engine, if_exists="replace", index=False)
    print("Loaded `Dictionary` table.")

    print("ETL process completed successfully!")
except Exception as e:
    print(f"ETL process failed: {e}")


Loaded `Stay` table.
ETL process completed successfully!
